In [38]:
# The code was removed by Watson Studio for sharing.

In [43]:
spark = SparkSession\
    .builder\
    .appName("Cloudant Spark SQL Example in Python using temp tables")\
    .config("cloudant.host",credentials_1['custom_url'].split('@')[1])\
    .config("cloudant.username", credentials_1['username'])\
    .config("cloudant.password",credentials_1['password'])\
    .config("jsonstore.rdd.partitions", 1)\
    .getOrCreate()

In [44]:
df=spark.read.load('shake_classification', "com.cloudant.spark")

df.createOrReplaceTempView("df")
spark.sql("SELECT * from df").show()

+-----+--------+-----+-----+-----+--------------------+--------------------+
|CLASS|SENSORID|    X|    Y|    Z|                 _id|                _rev|
+-----+--------+-----+-----+-----+--------------------+--------------------+
|    0|asdfghjk| 0.04| 0.04| 0.04|001fd4da2a1ced55e...|1-004320ad84dbdd4...|
|    0|asdfghjk|-0.04|-0.04|-0.04|001fd4da2a1ced55e...|1-3dfd90aea317f83...|
|    0|asdfghjk|  0.1|  0.1|  0.1|001fd4da2a1ced55e...|1-9ce895790043823...|
|    0|asdfghjk|  0.0|  0.0|  0.0|001fd4da2a1ced55e...|1-461daac6b553896...|
|    0|asdfghjk| 0.04| 0.04| 0.04|001fd4da2a1ced55e...|1-004320ad84dbdd4...|
|    0|asdfghjk|  0.0|  0.0|  0.0|001fd4da2a1ced55e...|1-461daac6b553896...|
|    0|asdfghjk| 0.01| 0.01| 0.01|001fd4da2a1ced55e...|1-a4b6e8d60269136...|
|    0|asdfghjk| 0.15| 0.15| 0.15|001fd4da2a1ced55e...|1-af099c11156019c...|
|    0|asdfghjk| 0.74| 0.74| 0.74|001fd4da2a1ced55e...|1-dd43f14a1a4bd9f...|
|    0|asdfghjk| -0.1| -0.1| -0.1|001fd4da2a1ced55e...|1-f1a6d8ac5db7570...|

In [49]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['X','Y','Z'], outputCol = 'features')

In [50]:
from pyspark.ml.classification import LogisticRegression

classifier = LogisticRegression(labelCol = 'CLASS',maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [51]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, classifier])

In [53]:
model = pipeline.fit(df)

In [54]:

prediction = model.transform(df)

In [55]:
prediction.show()

+-----+--------+-----+-----+-----+--------------------+--------------------+-------------------+--------------------+--------------------+----------+
|CLASS|SENSORID|    X|    Y|    Z|                 _id|                _rev|           features|       rawPrediction|         probability|prediction|
+-----+--------+-----+-----+-----+--------------------+--------------------+-------------------+--------------------+--------------------+----------+
|    0|asdfghjk| 0.04| 0.04| 0.04|001fd4da2a1ced55e...|1-004320ad84dbdd4...|   [0.04,0.04,0.04]|[-0.6734577544034...|[0.33772302463891...|       1.0|
|    0|asdfghjk|-0.04|-0.04|-0.04|001fd4da2a1ced55e...|1-3dfd90aea317f83...|[-0.04,-0.04,-0.04]|[-0.6734577544034...|[0.33772302463891...|       1.0|
|    0|asdfghjk|  0.1|  0.1|  0.1|001fd4da2a1ced55e...|1-9ce895790043823...|      [0.1,0.1,0.1]|[-0.6734577544034...|[0.33772302463891...|       1.0|
|    0|asdfghjk|  0.0|  0.0|  0.0|001fd4da2a1ced55e...|1-461daac6b553896...|          (3,[],[])|[-0.

In [57]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("CLASS")
    
binEval.evaluate(prediction)

0.6622769753610875

In [59]:

!rm -Rf a2_m2.json

In [60]:
prediction = prediction.repartition(1)
prediction.write.json('a2_m2.json')

In [61]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py

--2019-01-11 23:45:11--  https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2289 (2.2K) [text/plain]
Saving to: ‘rklib.py’

100%[======================================>] 2,289       --.-K/s   in 0s      

2019-01-11 23:45:11 (20.2 MB/s) - ‘rklib.py’ saved [2289/2289]



In [62]:
!zip -r a2_m2.json.zip a2_m2.json

  adding: a2_m2.json/ (stored 0%)
  adding: a2_m2.json/part-00000-ed42c71f-b636-496f-99cf-e5ff7158025b.json (deflated 91%)
  adding: a2_m2.json/_SUCCESS (stored 0%)
  adding: a2_m2.json/._SUCCESS.crc (stored 0%)
  adding: a2_m2.json/.part-00000-ed42c71f-b636-496f-99cf-e5ff7158025b.json.crc (stored 0%)


In [63]:
!base64 a2_m2.json.zip > a2_m2.json.zip.base64

In [64]:
from rklib import submit
key = "J3sDL2J8EeiaXhILFWw2-g"
part = "G4P6f"
email = 'ia11287n@pace.edu'
secret = 'qQWiYlKFqmlIGTrs'

with open('a2_m2.json.zip.base64', 'r') as myfile:
    data=myfile.read()
submit(email, secret, key, part, [part], data)

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"LTL4F","id":"f_F-qCtuEei_fRLwaVDk3g~LTL4F~fwy4MhYtEemPLArb-44eHA","courseId":"f_F-qCtuEei_fRLwaVDk3g"}],"paging":{},"linked":{}}
-------------------------
